In [5]:
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
from dummy import *
from GCN_model import *

dtype = torch.long
batch_size = 128
learning_rate = 0.0001
num_epochs = 50
n_nodes = 5
hidden_dim = 10
n_classes = 4
dropout = 0.5

train_dataset, valid_dataset = get_dataset(n_train=65536, n_valid=8192, n_nodes=n_nodes, n_classes=n_classes)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

valid_loader = torch.utils.data.DataLoader(dataset=valid_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)
X = torch.eye(n_nodes)

                             ##############

model = TwoLayerGCN(n_nodes, hidden_dim, n_classes, dropout)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (A, labels) in enumerate(train_loader):
        
        # Forward pass
        outputs = model(X, A)
        loss = criterion(outputs.transpose(1,2), labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 8 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss))

with torch.no_grad():
    correct = 0
    total = 0
    for A, labels in valid_loader:
        outputs = model(X, A)
        _, predicted = torch.max(outputs, 2)
        total += labels.numel()
        correct += (predicted == labels).sum()

    print('Accuracy of the network on the test data: {} %'.format(100 * correct / total))


Epoch [1/50], Step [8/512], Loss: 1.4032
Epoch [1/50], Step [16/512], Loss: 1.4010
Epoch [1/50], Step [24/512], Loss: 1.3981
Epoch [1/50], Step [32/512], Loss: 1.4001
Epoch [1/50], Step [40/512], Loss: 1.3928
Epoch [1/50], Step [48/512], Loss: 1.4008
Epoch [1/50], Step [56/512], Loss: 1.4066
Epoch [1/50], Step [64/512], Loss: 1.4149
Epoch [1/50], Step [72/512], Loss: 1.4058
Epoch [1/50], Step [80/512], Loss: 1.3979
Epoch [1/50], Step [88/512], Loss: 1.4042
Epoch [1/50], Step [96/512], Loss: 1.3950
Epoch [1/50], Step [104/512], Loss: 1.3919
Epoch [1/50], Step [112/512], Loss: 1.4010
Epoch [1/50], Step [120/512], Loss: 1.4056
Epoch [1/50], Step [128/512], Loss: 1.4070
Epoch [1/50], Step [136/512], Loss: 1.3916
Epoch [1/50], Step [144/512], Loss: 1.4074
Epoch [1/50], Step [152/512], Loss: 1.3864
Epoch [1/50], Step [160/512], Loss: 1.3974
Epoch [1/50], Step [168/512], Loss: 1.4108
Epoch [1/50], Step [176/512], Loss: 1.3897
Epoch [1/50], Step [184/512], Loss: 1.4007
Epoch [1/50], Step [192/

Epoch [4/50], Step [48/512], Loss: 1.3934
Epoch [4/50], Step [56/512], Loss: 1.3943
Epoch [4/50], Step [64/512], Loss: 1.3931
Epoch [4/50], Step [72/512], Loss: 1.3935
Epoch [4/50], Step [80/512], Loss: 1.3890
Epoch [4/50], Step [88/512], Loss: 1.3908
Epoch [4/50], Step [96/512], Loss: 1.3909
Epoch [4/50], Step [104/512], Loss: 1.3898
Epoch [4/50], Step [112/512], Loss: 1.3879
Epoch [4/50], Step [120/512], Loss: 1.3894
Epoch [4/50], Step [128/512], Loss: 1.3934
Epoch [4/50], Step [136/512], Loss: 1.3924
Epoch [4/50], Step [144/512], Loss: 1.3917
Epoch [4/50], Step [152/512], Loss: 1.3836
Epoch [4/50], Step [160/512], Loss: 1.3830
Epoch [4/50], Step [168/512], Loss: 1.3876
Epoch [4/50], Step [176/512], Loss: 1.3933
Epoch [4/50], Step [184/512], Loss: 1.3838
Epoch [4/50], Step [192/512], Loss: 1.3862
Epoch [4/50], Step [200/512], Loss: 1.3941
Epoch [4/50], Step [208/512], Loss: 1.3959
Epoch [4/50], Step [216/512], Loss: 1.3896
Epoch [4/50], Step [224/512], Loss: 1.3918
Epoch [4/50], Step

Epoch [7/50], Step [144/512], Loss: 1.3925
Epoch [7/50], Step [152/512], Loss: 1.3887
Epoch [7/50], Step [160/512], Loss: 1.3927
Epoch [7/50], Step [168/512], Loss: 1.3897
Epoch [7/50], Step [176/512], Loss: 1.3835
Epoch [7/50], Step [184/512], Loss: 1.3910
Epoch [7/50], Step [192/512], Loss: 1.3897
Epoch [7/50], Step [200/512], Loss: 1.3852
Epoch [7/50], Step [208/512], Loss: 1.3898
Epoch [7/50], Step [216/512], Loss: 1.3880
Epoch [7/50], Step [224/512], Loss: 1.3895
Epoch [7/50], Step [232/512], Loss: 1.3885
Epoch [7/50], Step [240/512], Loss: 1.3875
Epoch [7/50], Step [248/512], Loss: 1.3897
Epoch [7/50], Step [256/512], Loss: 1.3835
Epoch [7/50], Step [264/512], Loss: 1.3888
Epoch [7/50], Step [272/512], Loss: 1.3890
Epoch [7/50], Step [280/512], Loss: 1.3873
Epoch [7/50], Step [288/512], Loss: 1.3904
Epoch [7/50], Step [296/512], Loss: 1.3919
Epoch [7/50], Step [304/512], Loss: 1.3858
Epoch [7/50], Step [312/512], Loss: 1.3872
Epoch [7/50], Step [320/512], Loss: 1.3892
Epoch [7/50

Epoch [10/50], Step [168/512], Loss: 1.3863
Epoch [10/50], Step [176/512], Loss: 1.3859
Epoch [10/50], Step [184/512], Loss: 1.3875
Epoch [10/50], Step [192/512], Loss: 1.3841
Epoch [10/50], Step [200/512], Loss: 1.3854
Epoch [10/50], Step [208/512], Loss: 1.3852
Epoch [10/50], Step [216/512], Loss: 1.3880
Epoch [10/50], Step [224/512], Loss: 1.3879
Epoch [10/50], Step [232/512], Loss: 1.3881
Epoch [10/50], Step [240/512], Loss: 1.3867
Epoch [10/50], Step [248/512], Loss: 1.3863
Epoch [10/50], Step [256/512], Loss: 1.3876
Epoch [10/50], Step [264/512], Loss: 1.3874
Epoch [10/50], Step [272/512], Loss: 1.3890
Epoch [10/50], Step [280/512], Loss: 1.3868
Epoch [10/50], Step [288/512], Loss: 1.3859
Epoch [10/50], Step [296/512], Loss: 1.3866
Epoch [10/50], Step [304/512], Loss: 1.3860
Epoch [10/50], Step [312/512], Loss: 1.3900
Epoch [10/50], Step [320/512], Loss: 1.3857
Epoch [10/50], Step [328/512], Loss: 1.3880
Epoch [10/50], Step [336/512], Loss: 1.3880
Epoch [10/50], Step [344/512], L

Epoch [13/50], Step [224/512], Loss: 1.3876
Epoch [13/50], Step [232/512], Loss: 1.3894
Epoch [13/50], Step [240/512], Loss: 1.3868
Epoch [13/50], Step [248/512], Loss: 1.3873
Epoch [13/50], Step [256/512], Loss: 1.3874
Epoch [13/50], Step [264/512], Loss: 1.3835
Epoch [13/50], Step [272/512], Loss: 1.3848
Epoch [13/50], Step [280/512], Loss: 1.3855
Epoch [13/50], Step [288/512], Loss: 1.3881
Epoch [13/50], Step [296/512], Loss: 1.3862
Epoch [13/50], Step [304/512], Loss: 1.3868
Epoch [13/50], Step [312/512], Loss: 1.3878
Epoch [13/50], Step [320/512], Loss: 1.3865
Epoch [13/50], Step [328/512], Loss: 1.3844
Epoch [13/50], Step [336/512], Loss: 1.3859
Epoch [13/50], Step [344/512], Loss: 1.3864
Epoch [13/50], Step [352/512], Loss: 1.3879
Epoch [13/50], Step [360/512], Loss: 1.3860
Epoch [13/50], Step [368/512], Loss: 1.3882
Epoch [13/50], Step [376/512], Loss: 1.3872
Epoch [13/50], Step [384/512], Loss: 1.3871
Epoch [13/50], Step [392/512], Loss: 1.3883
Epoch [13/50], Step [400/512], L

...

Epoch [37/50], Step [200/512], Loss: 1.3864
Epoch [37/50], Step [208/512], Loss: 1.3868
Epoch [37/50], Step [216/512], Loss: 1.3864
Epoch [37/50], Step [224/512], Loss: 1.3868
Epoch [37/50], Step [232/512], Loss: 1.3870
Epoch [37/50], Step [240/512], Loss: 1.3868
Epoch [37/50], Step [248/512], Loss: 1.3860
Epoch [37/50], Step [256/512], Loss: 1.3862
Epoch [37/50], Step [264/512], Loss: 1.3864
Epoch [37/50], Step [272/512], Loss: 1.3859
Epoch [37/50], Step [280/512], Loss: 1.3862
Epoch [37/50], Step [288/512], Loss: 1.3864
Epoch [37/50], Step [296/512], Loss: 1.3860
Epoch [37/50], Step [304/512], Loss: 1.3863
Epoch [37/50], Step [312/512], Loss: 1.3865
Epoch [37/50], Step [320/512], Loss: 1.3868
Epoch [37/50], Step [328/512], Loss: 1.3865
Epoch [37/50], Step [336/512], Loss: 1.3862
Epoch [37/50], Step [344/512], Loss: 1.3855
Epoch [37/50], Step [352/512], Loss: 1.3866
Epoch [37/50], Step [360/512], Loss: 1.3857
Epoch [37/50], Step [368/512], Loss: 1.3857
Epoch [37/50], Step [376/512], L

Epoch [40/50], Step [176/512], Loss: 1.3867
Epoch [40/50], Step [184/512], Loss: 1.3871
Epoch [40/50], Step [192/512], Loss: 1.3859
Epoch [40/50], Step [200/512], Loss: 1.3864
Epoch [40/50], Step [208/512], Loss: 1.3862
Epoch [40/50], Step [216/512], Loss: 1.3861
Epoch [40/50], Step [224/512], Loss: 1.3862
Epoch [40/50], Step [232/512], Loss: 1.3858
Epoch [40/50], Step [240/512], Loss: 1.3861
Epoch [40/50], Step [248/512], Loss: 1.3864
Epoch [40/50], Step [256/512], Loss: 1.3858
Epoch [40/50], Step [264/512], Loss: 1.3868
Epoch [40/50], Step [272/512], Loss: 1.3861
Epoch [40/50], Step [280/512], Loss: 1.3853
Epoch [40/50], Step [288/512], Loss: 1.3857
Epoch [40/50], Step [296/512], Loss: 1.3865
Epoch [40/50], Step [304/512], Loss: 1.3865
Epoch [40/50], Step [312/512], Loss: 1.3866
Epoch [40/50], Step [320/512], Loss: 1.3868
Epoch [40/50], Step [328/512], Loss: 1.3852
Epoch [40/50], Step [336/512], Loss: 1.3863
Epoch [40/50], Step [344/512], Loss: 1.3864
Epoch [40/50], Step [352/512], L

Epoch [43/50], Step [144/512], Loss: 1.3867
Epoch [43/50], Step [152/512], Loss: 1.3866
Epoch [43/50], Step [160/512], Loss: 1.3870
Epoch [43/50], Step [168/512], Loss: 1.3866
Epoch [43/50], Step [176/512], Loss: 1.3862
Epoch [43/50], Step [184/512], Loss: 1.3856
Epoch [43/50], Step [192/512], Loss: 1.3860
Epoch [43/50], Step [200/512], Loss: 1.3863
Epoch [43/50], Step [208/512], Loss: 1.3860
Epoch [43/50], Step [216/512], Loss: 1.3857
Epoch [43/50], Step [224/512], Loss: 1.3861
Epoch [43/50], Step [232/512], Loss: 1.3864
Epoch [43/50], Step [240/512], Loss: 1.3856
Epoch [43/50], Step [248/512], Loss: 1.3864
Epoch [43/50], Step [256/512], Loss: 1.3861
Epoch [43/50], Step [264/512], Loss: 1.3865
Epoch [43/50], Step [272/512], Loss: 1.3863
Epoch [43/50], Step [280/512], Loss: 1.3867
Epoch [43/50], Step [288/512], Loss: 1.3865
Epoch [43/50], Step [296/512], Loss: 1.3864
Epoch [43/50], Step [304/512], Loss: 1.3859
Epoch [43/50], Step [312/512], Loss: 1.3865
Epoch [43/50], Step [320/512], L

Epoch [46/50], Step [208/512], Loss: 1.3855
Epoch [46/50], Step [216/512], Loss: 1.3858
Epoch [46/50], Step [224/512], Loss: 1.3865
Epoch [46/50], Step [232/512], Loss: 1.3867
Epoch [46/50], Step [240/512], Loss: 1.3858
Epoch [46/50], Step [248/512], Loss: 1.3868
Epoch [46/50], Step [256/512], Loss: 1.3862
Epoch [46/50], Step [264/512], Loss: 1.3856
Epoch [46/50], Step [272/512], Loss: 1.3862
Epoch [46/50], Step [280/512], Loss: 1.3862
Epoch [46/50], Step [288/512], Loss: 1.3864
Epoch [46/50], Step [296/512], Loss: 1.3870
Epoch [46/50], Step [304/512], Loss: 1.3861
Epoch [46/50], Step [312/512], Loss: 1.3864
Epoch [46/50], Step [320/512], Loss: 1.3864
Epoch [46/50], Step [328/512], Loss: 1.3862
Epoch [46/50], Step [336/512], Loss: 1.3856
Epoch [46/50], Step [344/512], Loss: 1.3861
Epoch [46/50], Step [352/512], Loss: 1.3860
Epoch [46/50], Step [360/512], Loss: 1.3866
Epoch [46/50], Step [368/512], Loss: 1.3862
Epoch [46/50], Step [376/512], Loss: 1.3859
Epoch [46/50], Step [384/512], L

Epoch [49/50], Step [232/512], Loss: 1.3864
Epoch [49/50], Step [240/512], Loss: 1.3864
Epoch [49/50], Step [248/512], Loss: 1.3865
Epoch [49/50], Step [256/512], Loss: 1.3861
Epoch [49/50], Step [264/512], Loss: 1.3867
Epoch [49/50], Step [272/512], Loss: 1.3858
Epoch [49/50], Step [280/512], Loss: 1.3862
Epoch [49/50], Step [288/512], Loss: 1.3865
Epoch [49/50], Step [296/512], Loss: 1.3864
Epoch [49/50], Step [304/512], Loss: 1.3861
Epoch [49/50], Step [312/512], Loss: 1.3863
Epoch [49/50], Step [320/512], Loss: 1.3867
Epoch [49/50], Step [328/512], Loss: 1.3857
Epoch [49/50], Step [336/512], Loss: 1.3859
Epoch [49/50], Step [344/512], Loss: 1.3859
Epoch [49/50], Step [352/512], Loss: 1.3860
Epoch [49/50], Step [360/512], Loss: 1.3862
Epoch [49/50], Step [368/512], Loss: 1.3865
Epoch [49/50], Step [376/512], Loss: 1.3867
Epoch [49/50], Step [384/512], Loss: 1.3868
Epoch [49/50], Step [392/512], Loss: 1.3862
Epoch [49/50], Step [400/512], Loss: 1.3861
Epoch [49/50], Step [408/512], L